In [ ]:
import asyncio
import nest_asyncio
from atproto import Client
from confluent_kafka import Producer
import json

# Permitir loops en Jupyter
nest_asyncio.apply()

# Configuración de Bluesky
BLUESKY_HANDLE = 'l3chu6a.bsky.social'  # Reemplazar con tu usuario de Bluesky
BLUESKY_PASSWORD = 'Bluesky_005'  # Reemplazar con tu contraseña de Bluesky

# Configuración de Kafka
kafka_conf = {'bootstrap.servers': 'kafka:9092'}
producer = Producer(kafka_conf)
topic = 'bluesky-feed'

# Función para enviar a Kafka
def send_to_kafka(key, value):
    producer.produce(topic, key=key, value=value)
    producer.flush()

# Función para obtener y procesar publicaciones de Bluesky
# async def get_bluesky_posts():
#     client = Client()
    
#     # Login a Bluesky
#     await client.login(BLUESKY_HANDLE, BLUESKY_PASSWORD)
    
#     # Suscribirse a eventos de publicaciones (feed)
#     async for event in client.subscribe_feeds():
#         # Procesar solo publicaciones del tipo "create"
#         if event['type'] == 'commit':
#             ops = event['ops']
#             for op in ops:
#                 if op['action'] == 'create' and op['path'].startswith('app.bsky.feed.post'):
#                     post_data = op['value']
#                     text = post_data.get('text')
#                     if text:
#                         # Preparar datos y enviarlos a Kafka
#                         key = event['repo']
#                         value = json.dumps({'author': key, 'text': text})
#                         send_to_kafka(key, value)
#                         print(f"Enviado a Kafka: {value}")

async def get_bluesky_posts():
    client = Client()
    
    # Login a Bluesky
    await client.login(BLUESKY_HANDLE, BLUESKY_PASSWORD)
    
    # Suscribirse a eventos de publicaciones (feed)
    async for event in client.subscribe_feeds():
        if isinstance(event, dict):  # Asegúrate de que el event sea un diccionario
            # Procesar solo publicaciones del tipo "create"
            if event.get('type') == 'commit':
                ops = event.get('ops', [])
                for op in ops:
                    if op.get('action') == 'create' and op['path'].startswith('app.bsky.feed.post'):
                        post_data = op.get('value', {})
                        text = post_data.get('text')
                        if text:
                            key = event.get('repo')
                            value = json.dumps({'author': key, 'text': text})
                            send_to_kafka(key, value)
                            print(f"Enviado a Kafka: {value}")



# Ejecutar el loop de eventos
await get_bluesky_posts()
# asyncio.run(get_bluesky_posts())

